# 1. Installs, Settings, Imports

In [1]:
#install pyDF2
#!pip install PyPDF2

In [2]:
# importing all the required modules
import PyPDF2
from PyPDF2 import PdfReader
import os, glob, pandas as pd, re

In [3]:
pd.set_option('display.max_columns', None)  # or 1000
pd.set_option('display.max_rows', None)  # or 1000
pd.set_option('display.max_colwidth', None)  # or 199

# 1.Import data 

In [4]:
files_info = []
for filename in glob.iglob('data/**/*.pdf',
                           recursive = True):
    try:
        reader = PdfReader(filename)
        text = ""
        for page in reader.pages:
            text += page.extract_text() + "\n"
        files_info.append([filename, text])
    except:
        print(filename)
    
print(len(files_info))

unknown widths : 
[0, IndirectObject(242, 0, 2532839972384)]
unknown widths : 
[0, IndirectObject(245, 0, 2532839972384)]
unknown widths : 
[0, IndirectObject(248, 0, 2532839972384)]
unknown widths : 
[0, IndirectObject(251, 0, 2532839972384)]
unknown widths : 
[0, IndirectObject(254, 0, 2532839972384)]
unknown widths : 
[0, IndirectObject(257, 0, 2532839972384)]
unknown widths : 
[0, IndirectObject(260, 0, 2532839972384)]
unknown widths : 
[0, IndirectObject(263, 0, 2532839972384)]
unknown widths : 
[0, IndirectObject(266, 0, 2532839972384)]
unknown widths : 
[0, IndirectObject(269, 0, 2532839972384)]
unknown widths : 
[0, IndirectObject(272, 0, 2532839972384)]
unknown widths : 
[0, IndirectObject(275, 0, 2532839972384)]
unknown widths : 
[0, IndirectObject(278, 0, 2532839972384)]
unknown widths : 
[0, IndirectObject(281, 0, 2532839972384)]


919


In [6]:
df = pd.DataFrame(files_info, columns = ['Filename', 'Text'])

In [7]:
print(len(df))

919


# 2. Adding type

In [10]:
df["Type"] = df["Filename"].apply(lambda x: "advertencia" if x.startswith("data\\advertencia_PDF") else "seguimiento")

# 3. Adding Year

In [19]:
def get_year(filename):
    filenames = ["data\\seguimiento\\", "data\\advertencia_PDF\\AT ", "data\\advertencia_PDF\\IR "]
    for x in filenames:
        if filename.startswith(x):
            return filename.split(x)[1][:4]
        
df["Year"] = df["Filename"].apply(get_year)

In [18]:
s = "data\\advertencia_PDF\\AT 2018\\AT N° 001-18 CAU-Suárez.pdf"
s.split("data\\advertencia_PDF\\AT ")[1][:4]

'2018'

In [21]:
df.Year.unique()

array(['2018', '2019', '2020', '2021', '2022', '2001', '2002', '2003',
       '2004', '2005', '2006', '2007', '2008', '2009', '2010', '2011',
       '2012', '2013'], dtype=object)

# 4. Cleaning Text

In [25]:
import string
replacing = [
    "Línea Nacional 018000914814 BUREAU VERrTAS Cenificationwwwdefensoriagovco Plantilla VIgente desde 27062017",
    "Carrera 8No 12B31 Bogotá DC",
    "Doctora NANCY PATRICIA GUTIERREZ CASTAÑEDA Ministerio del Interior Secretaría Técnica de laComisión Intersectorial para la Respuesta Rápida alas Alertas Tempranas CIPRAT",
    "Doctora NANCY PATRICIA GUTIERREZ CASTAÑEDA ·Ministerio del Interior Secretaría Técnica de la Comisión Intersectorial para la Respuesta Rápida alas Alertas Tempranas CIPRAT",
    "Doctora NANCY PATRICIA GUTIERREZ CASTAÑEDA ·Ministerio del Interior Secretaría Técnica de laComisión Intersectorial para la Respuesta Rápida alas Alertas Tempranas CIPRAT",
    "Sede Central Calle 55 No 1032 Tel 3147300 Ext fax 2452 Bogotá DC Colombia correo electrónico satdefensoriaorgco",
    "Doctor GUILLERMO RIVERA FLÓREZ Ministro del Interior Secretaría Técnica de la Comisión Intersectorial para la Respuesta Rápida a las Alertas Tempranas",
    "Sede Central: Calle 55 No. 10 -32 Tels: 3147300 Ext. 2437 Telefax 3147300 Ext. 2452 Bogotá, D.C., Colombia correo electrónico: sat@defensoria.org.co",
    "Sede Central Calle 55 N° 10-32 Teléfono 3147300 Ext 2437 Fax:3147300 Ext 2452 Correo electrónico: sat@defensoria.org.co",
    "Sede Central: Calle 55 No. 10-32 Teléfono: 3147300 ex 2437 Telefax 3147300 ext 2452 Correo Electrónico: sat@defensoria.org.co - Bogotá, D.C., Colombia", 
    "Sede Central: Calle 55 No. 10-32 Teléfono: 3147300 Ext. 2437 Telefax 3147300 ext 2452 Correo Electrónico: sat@defensoria.org.co - Bogotá, D.C., Colombia", 
    "Sede Central: Calle 55 No. 10-32 Teléfonos: 6915300 ; 3147300 extensión: 2437, fax ext. 2452 Correo Electrónico: sat@defensoria.org.co – Calle 55 No. 10 -32 Bogotá, D.C. Colombia",
             "Carrera 9 # 16 -21 Bogotá D.C. PBX: (57) (1) 3147300 Línea Nacional: 01 8000 914814 www.defensoria.gov.co Plantilla Vigente desde:",
             "Doctor: GUILLERMO RIVERA FLÓREZ Ministro del Interior Secretaría Técnica de la Comisión Intersectorial para la Respuesta Rápida a las Alertas Tempranas (CIPRAT) Carrera 8 No. 12 B - 31 BOGOTÁ, D.C. Referencia:",
             "CARLOS ALFONSO NEGRET MOSQUERA Defensor del Pueblo JProyectó: Mariana Hernández ArandaQ Revisó: Fernando Gaitán Peña Aprobó: Mauricio Redondo Valencia Consecutivo Dependencia: '7,n~\"-..11 .. 5Sentencia 00463 de 2018 Consejo de Estado c.p Jaime Orlando Santofimio Gamboa \":\" \". .Línea Nacional: 01 8000914814ISO9001: 2008 NTC GP 1000: 2009 www.defensoria.goY.co Plantilla Vigente desde: 27106/2017BUREAU VERITAS Certiticatíon", 
             "Carrera 9 # 16 -21, Bogo tá, D.C. PBX: (57) ( 1) 314 73 00, ext .: 3402 · Línea Nacional: 01 8000 914814 www.defensoria.gov.co Código: AD -P06 -F-20 - VERSIÓN: 04 - vigente desde: 07/12 /2016"]

def cleantext(text):
    text = text.strip()
    if text == "" or text != text or text == " ":
        return ""
    for s in replacing:
        if s in text:
            print("*")
            text = text.replace(s, "")
    text = text.translate(str.maketrans('', '', string.punctuation))
    
    return text

df["Text"] = df["Text"].apply(cleantext)

# 4. Saving text in csv

In [26]:

df.to_csv("data/alertas.csv", sep="|", index=False)

# 5. Saving HTML

In [27]:
!jupyter nbconvert --to html 1_Parsing_Data.ipynb

[NbConvertApp] Converting notebook 1_Parsing_Data.ipynb to html
[NbConvertApp] Writing 581604 bytes to 1_Parsing_Data.html
